In [1]:
!pip install torchinfo

import torch
import torch.nn as nn
import requests
import math
import pathlib as path
import os
import zipfile
import torchvision
import torchvision.transforms as transforms
import torchinfo

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:
train_set = torchvision.datasets.CIFAR10(
    root = "/content/",
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)

test_set = torchvision.datasets.CIFAR10(
    root='/content',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)



Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size = 128,
    shuffle = True,
    num_workers = os.cpu_count()
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size = 128,
    shuffle = False,
    num_workers = os.cpu_count()
)

In [4]:
sample = next(iter(train_loader))
sample[0][0].shape

torch.Size([3, 32, 32])

In [5]:
class Image_Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 20, 3, 1, 1),
        nn.ReLU(),
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.Conv2d(20, 20, 3, 1, 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(20*4*4, 10),
    )
  def forward(self, x):
    return self.classifier(self.layer3(self.layer2(self.layer1(x))))

def accuracy(y_pred, y):
  correct = torch.eq(y_pred, y).sum().item()
  return (correct/len(y))*100

In [6]:
model = Image_Model().to(device)

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)


In [12]:
torch.manual_seed(42)
epochs = 5
for epoch in range(epochs):
  for batch,(x,y) in enumerate(train_loader):
    model.train()
    logits = model(x.to(device))
    y_pred = torch.argmax(torch.softmax(logits, dim=1), dim=1)
    loss = loss_fn(logits, y.to(device))
    acc = accuracy(y_pred, y.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #Testing

    model.eval()
    with torch.inference_mode():
      test_logits = model(x.to(device))
      test_y_pred = torch.argmax(torch.softmax(test_logits, dim=1), dim=1)
      test_loss = loss_fn(test_logits, y.to(device))
      test_acc = accuracy(test_y_pred, y.to(device))
      if batch%100 == 0:
        print(f"Epoch: {epoch} Batch: {batch} train_Loss: {loss.item()} Test_loss: {test_loss} Test_Accuracy: {test_acc}")

Epoch: 0 Batch: 0 train_Loss: 2.3023388385772705 Test_loss: 2.302252769470215 Test_Accuracy: 13.28125
Epoch: 0 Batch: 100 train_Loss: 2.3025550842285156 Test_loss: 2.3024771213531494 Test_Accuracy: 11.71875
Epoch: 0 Batch: 200 train_Loss: 2.303398609161377 Test_loss: 2.303302049636841 Test_Accuracy: 7.03125
Epoch: 0 Batch: 300 train_Loss: 2.3025310039520264 Test_loss: 2.3023860454559326 Test_Accuracy: 9.375
Epoch: 1 Batch: 0 train_Loss: 2.302516460418701 Test_loss: 2.302385091781616 Test_Accuracy: 10.15625
Epoch: 1 Batch: 100 train_Loss: 2.302142858505249 Test_loss: 2.3019232749938965 Test_Accuracy: 8.59375
Epoch: 1 Batch: 200 train_Loss: 2.302014112472534 Test_loss: 2.301844358444214 Test_Accuracy: 10.9375
Epoch: 1 Batch: 300 train_Loss: 2.302838087081909 Test_loss: 2.302672863006592 Test_Accuracy: 7.03125
Epoch: 2 Batch: 0 train_Loss: 2.302638053894043 Test_loss: 2.302551746368408 Test_Accuracy: 9.375
Epoch: 2 Batch: 100 train_Loss: 2.30234956741333 Test_loss: 2.3021767139434814 Test